Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%tensorflow_version 1.x
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'drive/My Drive/Colab Notebooks/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  regulizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + 0.01*regulizer)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.899895
Minibatch accuracy: 6.2%
Validation accuracy: 9.6%
Minibatch loss at step 500: 0.867755
Minibatch accuracy: 76.6%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 0.703149
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.941300
Minibatch accuracy: 73.4%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.839585
Minibatch accuracy: 78.9%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.860183
Minibatch accuracy: 78.1%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 0.806593
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Test accuracy: 87.7%


# 1-layer NN

In [0]:
def multilayer_perceptron(ds, weights, biases):
  layer_1 = tf.matmul(ds, weights['h1']) + biases['h1']
  layer_1 = tf.nn.relu(layer_1)
  logits = tf.matmul(layer_1, weights['out']) + biases['out']
  return logits

hidden_layer = 1024
batch_size = 128
graph = tf.Graph()

with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights = {
      'h1': tf.Variable(tf.random_normal([image_size*image_size, hidden_layer])),
      'out': tf.Variable(tf.random_normal([hidden_layer, num_labels]))
  }

  biases = {
      'h1': tf.Variable(tf.zeros([hidden_layer])),
      'out': tf.Variable(tf.zeros([num_labels])) 
  }

  logits = multilayer_perceptron(tf_train_dataset, weights, biases)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  regulizers = tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])
  loss = tf.reduce_mean(loss + 0.01*regulizers)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron(test_dataset, weights, biases))

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4500.260254
Minibatch accuracy: 9.4%
Validation accuracy: 32.6%
Minibatch loss at step 500: 27.294937
Minibatch accuracy: 82.8%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 0.854555
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 0.851745
Minibatch accuracy: 76.6%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 0.807461
Minibatch accuracy: 78.1%
Validation accuracy: 83.9%
Minibatch loss at step 2500: 0.782702
Minibatch accuracy: 79.7%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 0.791460
Minibatch accuracy: 82.8%
Validation accuracy: 84.1%
Test accuracy: 89.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [0]:
num_steps = 3001

train_dataset_2 = train_dataset[:500,:]
train_labels_2 = train_labels[:500,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_2[offset:(offset + batch_size), :]
    batch_labels = train_labels_2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4509.837891
Minibatch accuracy: 10.2%
Validation accuracy: 38.5%
Minibatch loss at step 500: 27.216579
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 0.485874
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 0.275697
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 0.265946
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 2500: 0.255457
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 0.250052
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Test accuracy: 85.3%


Due to the limited number of examples in the training set, the model is able to learn it very well (100% minibatch accuracy), but the validation accuracy is really low. That is a sign of overfitting!

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [0]:
def multilayer_perceptron(ds, weights, biases):
  layer_1 = tf.matmul(ds, weights['h1']) + biases['h1']
  layer_1 = tf.nn.relu(layer_1)
  logits = tf.matmul(layer_1, weights['out']) + biases['out']
  return logits

hidden_layer = 1024
batch_size = 128
graph = tf.Graph()

with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights = {
      'h1': tf.Variable(tf.random_normal([image_size*image_size, hidden_layer])),
      'out': tf.Variable(tf.random_normal([hidden_layer, num_labels]))
  }

  biases = {
      'h1': tf.Variable(tf.zeros([hidden_layer])),
      'out': tf.Variable(tf.zeros([num_labels])) 
  }

  layer_1 = tf.matmul(tf_train_dataset, weights['h1']) + biases['h1']
  relu_layer_1 = tf.nn.relu(layer_1)
  dropout_layer_1 = tf.nn.dropout(relu_layer_1, rate=0.5)
  logits = tf.matmul(layer_1, weights['out']) + biases['out']

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  regulizers = tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])
  loss = tf.reduce_mean(loss + 0.01*regulizers)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron(test_dataset, weights, biases))

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4629.563477
Minibatch accuracy: 10.2%
Validation accuracy: 41.4%
Minibatch loss at step 500: 26.688364
Minibatch accuracy: 75.8%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 0.921974
Minibatch accuracy: 79.7%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.993358
Minibatch accuracy: 75.8%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 0.894462
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 0.936820
Minibatch accuracy: 78.1%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.854817
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Test accuracy: 86.7%


###Extreme overfitting

In [0]:
num_steps = 3001

train_dataset_2 = train_dataset[:500,:]
train_labels_2 = train_labels[:500,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_2[offset:(offset + batch_size), :]
    batch_labels = train_labels_2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4554.845703
Minibatch accuracy: 16.4%
Validation accuracy: 31.9%
Minibatch loss at step 500: 27.337843
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 1000: 0.444055
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 1500: 0.256356
Minibatch accuracy: 100.0%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 0.266266
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 0.256901
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 0.256145
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Test accuracy: 83.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [0]:
import math as math

def two_layer_train(ds, weights, biases):
  layer_1 = tf.matmul(ds, weights['h1']) + biases['h1']
  relu_layer_1 = tf.nn.relu(layer_1)

  layer_2 = tf.matmul(relu_layer_1, weights['h2']) + biases['h2']
  relu_layer_2 = tf.nn.relu(layer_2)

  layer_3 = tf.matmul(relu_layer_2, weights['h3']) + biases['h3']
  relu_layer_3 = tf.nn.relu(layer_3)

  layer_4 = tf.matmul(relu_layer_3, weights['h4']) + biases['h4']
  relu_layer_4 = tf.nn.relu(layer_4)
  logits = tf.matmul(relu_layer_4, weights['out']) + biases['out']
  return logits


hidden_layer_1 = 4096
hidden_layer_2 = 2048
hidden_layer_3 = 512
hidden_layer_4 = 256

batch_size = 256

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size,image_size*image_size])
  tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels])

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights = {
      'h1': tf.Variable(tf.truncated_normal([image_size*image_size, hidden_layer_1], stddev=math.sqrt(2.0/(image_size*image_size)))),
      'h2': tf.Variable(tf.truncated_normal([hidden_layer_1, hidden_layer_2], stddev=math.sqrt(2.0/(image_size*image_size)))),
      'h3': tf.Variable(tf.truncated_normal([hidden_layer_2, hidden_layer_3], stddev=math.sqrt(2.0/(image_size*image_size)))),
      'h4': tf.Variable(tf.truncated_normal([hidden_layer_3, hidden_layer_4], stddev=math.sqrt(2.0/(image_size*image_size)))),
      'out': tf.Variable(tf.truncated_normal([hidden_layer_4, num_labels], stddev=math.sqrt(2.0/(image_size*image_size))))
  }

  biases = {
      'h1': tf.Variable(tf.zeros([hidden_layer_1])),
      'h2': tf.Variable(tf.zeros([hidden_layer_2])),
      'h3': tf.Variable(tf.zeros([hidden_layer_3])),
      'h4': tf.Variable(tf.zeros([hidden_layer_4])),
      'out': tf.Variable(tf.zeros([num_labels]))
  }

  layer_1 = tf.matmul(tf_train_dataset, weights['h1']) + biases['h1']
  relu_layer_1 = tf.nn.relu(layer_1)

  layer_2 = tf.matmul(relu_layer_1, weights['h2']) + biases['h2']
  relu_layer_2 = tf.nn.relu(layer_2)

  layer_3 = tf.matmul(relu_layer_2, weights['h3']) + biases['h3']
  relu_layer_3 = tf.nn.relu(layer_3)

  layer_4 = tf.matmul(relu_layer_3, weights['h4']) + biases['h4']
  relu_layer_4 = tf.nn.relu(layer_4)

  logits = tf.matmul(relu_layer_4, weights['out']) + biases['out']

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  weights_reg = [tf.nn.l2_loss(weigth) for weigth in weights.values()]
  biases_reg = [tf.nn.l2_loss(bias) for bias in biases.values()]
  regulizers = sum(weights_reg) + sum(biases_reg)
  loss = tf.reduce_mean(loss + 0.001*regulizers)

  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(two_layer_train(valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(two_layer_train(test_dataset, weights, biases))

In [25]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.114830
Minibatch accuracy: 7.4%
Validation accuracy: 8.6%
Minibatch loss at step 500: 12.967811
Minibatch accuracy: 87.1%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 12.950684
Minibatch accuracy: 84.0%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 12.725400
Minibatch accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 12.534329
Minibatch accuracy: 87.9%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 12.404662
Minibatch accuracy: 91.0%
Validation accuracy: 86.9%
Minibatch loss at step 3000: 12.394123
Minibatch accuracy: 86.3%
Validation accuracy: 87.2%
Minibatch loss at step 3500: 12.183247
Minibatch accuracy: 87.1%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 12.223462
Minibatch accuracy: 83.6%
Validation accuracy: 87.8%
Minibatch loss at step 4500: 11.897497
Minibatch accuracy: 89.8%
Validation accuracy: 88.3%
Minibatch loss at step 5000: 11.770333
Minibatch accuracy: 89.8%
Validatio